<style>
.wzor {
  /* width: auto; */
  /* padding: auto; */
  border: 0.5rem solid gray;
  /* margin: 0; */
  background-color: lightgray;
}
</style>


# Modele analizy danych

### 2024/2025, semestr zimowy
Tomasz Rodak

---

## Wykład VIII

ISLP, rozdział 4.4

---

## Literatura
   1. James, G., Witten, D., Hastie, T., Tibshirani, R., & Taylor,
      J. (2023). An Introduction to Statistical Learning: With
      Applications in Python.
      
      [ISLP](https://www.statlearning.com/)
   
   2. Bishop, C. M., & Nasrabadi, N. M. (2006). Pattern recognition and
      machine learning (Vol. 4, No. 4, p. 738). New York: springer. 
   
      [PRML](https://www.microsoft.com/en-us/research/people/cmbishop/prml-book/)
   
   3. Kroese, D. P., Botev, Z., Taimre, T., & Vaisman, R. (2019). Data
      science and machine learning: mathematical and statistical
      methods. CRC Press.
   
      [https://people.smp.uq.edu.au/DirkKroese/DSML/](https://people.smp.uq.edu.au/DirkKroese/DSML/)
   
   4. Hastie, T., Tibshirani, R., Friedman, J. H., & Friedman,
      J. H. (2009). The elements of statistical learning: data mining,
      inference, and prediction (Vol. 2, pp. 1-758). New York:
      springer.
   
      [https://hastie.su.domains/ElemStatLearn/](https://hastie.su.domains/ElemStatLearn/)
   
   5. Murphy, K. P. (2022). Probabilistic machine learning: an
      introduction. MIT press.
   
      [https://probml.github.io/pml-book/book1.html](https://probml.github.io/pml-book/book1.html)
   
   6. Murphy, K. P. (2023). Probabilistic machine learning: Advanced
      topics. MIT press.
   
      [https://probml.github.io/pml-book/book2.html](https://probml.github.io/pml-book/book2.html)
   
   7. Bishop, C. M., & Bishop, H (2024). Deep learning. Springer.
   
      [https://www.bishopbook.com/](https://www.bishopbook.com/)
   
   8. Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep learning. MIT press.
   
      [https://www.deeplearningbook.org/](https://www.deeplearningbook.org/)

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis as LDA,
    QuadraticDiscriminantAnalysis as QDA
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

## Modele generatywne w problemach klasyfikacji: QDA, Naive Bayes

Przpomnijmy wprowadzony w poprzednim wykładzie schemat modelu generatywnego w problemie klasyfikacji. Niech $X$ będzie wektorem losowym cech, a $Y\in\{1,\ldots,K\}$ zmienną losową oznaczającą klasę. Na podstawie danych modelujemy rozkład a priori $P(Y=k)$ występowania oraz rozkłady warunkowe $p(x|Y=k)$ cech dla każdej klasy $k$ z osobna. Gdy te rozkłady już mamy, to możemy obliczyć rozkład a posteriori $P(Y=k|X=x)$ ze wzoru Bayesa:

$$
P(Y=k|x) = \frac{P(Y=k)p(x|Y=k)}{\sum_{k=1}^K P(Y=k)p(x|Y=k)}.
$$

Tak utworzony model pozwala na zdefiniowanie dyskryminatora w postaci klasyfikatora Bayesa: obserwacji $x$ przypisujemy tę klasę $k$, dla której $P(Y=k|x)$ jest największe oraz na generowanie nowych obserwacji $(x,y)$ zgodnie z rozkładem łącznym $p(x, y) = p(x|Y=k)P(Y=k)$.

Rozkłady a priori $P(Y=k)$ estymujemy na podstawie częstości występowania klas w zbiorze uczącym. Estymacja rozkładów warunkowych $p(x|Y=k)$ jest zadaniem trudniejszym i wymaga zastosowania dodatkowych założeń. W omówionym w poprzednim wykładzie modelu LDA zakładaliśmy, że rozkłady warunkowe $p(x|Y=k)$ są rozkładami normalnymi o jednakowych macierzach kowariancji. Mamy zatem założenie o postaci rozkładów (są to rozkłady normalne) i założenie o parametrach (macierz kowariancji jest taka sama dla wszystkich klas). W tym wykładzie omówimy kolejne dwa modele generatywne, QDA i Naive Bayes, które mają inne założenia o postaci rozkładów warunkowych $p(x|Y=k)$.


### Quadratic Discriminant Analysis (QDA)

W modelu QDA, podobnie jak w LDA, zakładamy, że rozkłady warunkowe $p(x|Y=k)$ są rozkładami normalnymi. Jednak w przeciwieństwie do LDA, nie zakładamy, że macierze kowariancji są jednakowe dla wszystkich klas. Mamy zatem założenie o postaci rozkładów (są to rozkłady normalne) i założenie o parametrach (macierz kowariancji może być inna dla każdej klasy). W modelu QDA rozkład warunkowy $p(x|Y=k)$ jest rozkładem normalnym o gęstości

$$
p(x|Y=k) = \frac{1}{(2\pi)^{p/2}|\Sigma_k|^{1/2}}\exp\left(-\frac{1}{2}(x-\mu_k)^T\Sigma_k^{-1}(x-\mu_k)\right),
$$

gdzie $\mu_k$ jest wektorem średnich cech dla klasy $k$, a $\Sigma_k$ jest macierzą kowariancji cech dla klasy $k$. Rozkład a priori $P(Y=k)$ estymujemy na podstawie częstości występowania klas w zbiorze uczącym. 

Licząc analogicznie jak w przypadku modelu LDA, otrzymujemy funkcje dyskryminacyjne postaci

$$
\delta_k(x) = -\frac{1}{2}(x-\mu_k)^T\Sigma_k^{-1}(x-\mu_k) - \frac{1}{2}\log|\Sigma_k| + \log P(Y=k).
$$

Klasyfikator Bayesa przypisuje obserwacji $x$ tę klasę $k$, dla której $\delta_k(x)$ jest największe. Funkcje są bardziej skomplikowane niż w przypadku LDA, ponieważ macierze kowariancji $\Sigma_k$ zależą od klasy $k$ i nie skracają się podczas upraszczania wyrażeń. Ponadto stopień tych funkcji względem $x$ jest dwa (w LDA był jeden) i dlatego model ten nazywamy *Quadratic Discriminant Analysis*. W konsekwencji granice decyzyjne w modelu QDA są zbudowane z fragmentów hiperpowierzchni stopnia drugiego (w zależnosci od wymiaru $p$ są to parabole, hiperbole, elipsy, paraboloidy, hiperboloidy, elipsoidy, itd.). 

Szacowanie parametrów modelu QDA jest zadaniem obliczeniowo trudniejszym niż w przypadku LDA, ponieważ musimy estymować $K$ macierzy kowariancji $\Sigma_k$ zamiast jednej macierzy kowariancji $\Sigma$ w przypadku LDA. Oznacza to konieczność obliczenia znacznie większej liczby parametrów. Z drugiej strony, model QDA jest mniej restrykcyjny niż LDA. Jeśli spodziewamy się nieliniowej granicy decyzyjnej, to model QDA może być lepszym wyborem niż LDA, gdyż da wtedy mniejszy *bias* (błąd obciążenia) niż LDA. LDA może w tym przypadku skutkować modelem niedouczonym. Jeśli jednak spodziewamy się, że granica decyzyjna jest liniowa, to model QDA będzie miał większy *variance* (błąd wariancji) niż LDA, ponieważ będzie miał więcej parametrów do dopasowania. W tym przypadku model QDA może skutkować modelem przeuczonym.

### Naiwny klasyfikator Bayesa (*Naive Bayes*)

W modela LDA i QDA zakładaliśmy, że rozkłady warunkowe $p(x|Y=k)$ są rozkładami normalnymi. W obu tych przypadkach szukaliśmy, przy założeniu, że znana jest klasa $k$, rozkładów *łącznych* wektorów losowych $X$. Oznacza to, że szukając rozkładu $X|Y=k$ estymujemy nie tylko rozkłady poszczególnych składowych wektora cech $X_j|Y=k$, ale także kowariancje między nimi. Problem ten ulega znacznemu uproszczeniu w naiwnym klasyfikatorze Bayesa (*Naive Bayes*), w którym zakładamy, że cechy $X_j$ są niezależne, gdy znamy klasę $k$. Wówczas warunkowy rozkład łączny $X|Y=k$ jest iloczynem rozkładów warunkowych poszczególnych składowych $X_j|Y=k$:

$$
p(x|Y=k) = \prod_{j=1}^p p(x_j|Y=k).
$$

Założenie o niezależności cech jest bardzo silne i rzadko jest spełnione w rzeczywistych problemach. Jednak w wielu przypadkach, nawet gdy to założenie nie jest spełnione naiwny klasyfikator Bayesa daje dobre wyniki. 

Rozkład a posteriori ma wówczas postać:

$$
P(Y=k|x) = \frac{P(Y=k)\prod_{j=1}^p p(x_j|Y=k)}{\sum_{k=1}^K P(Y=k)\prod_{j=1}^p p(x_j|Y=k)}.
$$

Aby zakończyć budowę modelu musimy zdefiniować rozkłady warunkowe $p(x_j|Y=k)$ dla każdej cechy $X_j$ i każdej klasy $k$. W tym celu możemy wykorzystać dowolny rozkład prawdopodobieństwa. Najczęściej stosuje się rozkłady normalne, Bernoulliego lub wielomianowe. 

* Jeśli cecha $X_j$ jest zmienną losową ciągłą, to możemy przyjąć, że $(X_j|Y=k)\sim N(\mu_{jk}, \sigma_{jk}^2)$, gdzie $\mu_{jk}$ i $\sigma_{jk}^2$ są parametrami rozkładu normalnego dla cechy $X_j$ i klasy $k$. Wówczas rozkład warunkowy $p(x|Y=k)$ jest rozkładem normalnym o gęstości $N(\mu_k, \Sigma_k)$, gdzie $\mu_k = (\mu_{1k}, \ldots, \mu_{pk})^T$ jest wektorem średnich cech dla klasy $k$, a $\Sigma_k$ jest **diagonalną** macierzą kowariancji cech dla klasy $k$ (wymaga obliczenia $p$ parametrów $\sigma_{jk}^2$ na głównej przekątnej, poza nią macierz jest zerowa).
* Inna opcja dla zmiennej losowej ciągłej to zdefiniowanie rozkładu $p(x_j|Y=k)$ na podstawie histogramu lub estymatora jądrowego. 
* Jeśli cecha $X_j$ jest kategoryjna (ma skończony zbiór wartości), to prawdopoobieństwo $p(x_j|Y=k)$ możemy estymować na podstawie częstości występowania poszczególnych wartości cechy $X_j$ w klasie $k$.


## Metody klasyfikacji - porównanie

Ustalmy klasę $K$ jako punkt odniesienia i rozważmy logarytmy stosunków prawdopodobieństw:

$$
\log\frac{P(Y=k|x)}{P(Y=K|x)},\qquad k=1,\ldots,K.
$$

Obserwację $x$ przypisujemy do klasy $k$, dla której logarytm ten jest największy. W przypadku modelu LDA mamy

\begin{align*}
\log\frac{P(Y=k|x)}{P(Y=K|x)} &= \log\frac{P(Y=k)}{P(Y=K)} + \log\frac{p(x|Y=k)}{p(x|Y=K)} \\
&= \log\frac{P(Y=k)}{P(Y=K)} - \frac{1}{2}(\mu_k+\mu_K)^T\Sigma^{-1}(\mu_k-\mu_K) + x^T\Sigma^{-1}(\mu_k-\mu_K)\\
&= a_k +\sum_{j=1}^p b_{kj}x_j,
\end{align*}

gdzie $a_k$ i $b_{kj}$ są stałymi niezależnymi od $x$. Zatem logarytmy szans w modelu LDA są funkcjami liniowymi względem $x$ dokładnie tak samo jak w przypadku regresji logistycznej. Różnica między tymi modelami polega na tym, że w regresji logistycznej szukamy parametrów $a_k$ i $b_{kj}$ bezpośrednio, natomiast w modelu LDA obliczamy najpierw parametry $P(Y=k)$,
$\mu_k$ i $\Sigma$ i dopiero na ich podstawie uzyskujemy parametry $a_k$ i $b_{kj}$. Zatem w modelu regresji logistycznej mamy mniej parametrów do dopasowania niż w modelu LDA. Ponadto w tym pierwszym przypadku nic nie zakładamy na temat postaci rozkładów warunkowych $p(x|Y=k)$ modelując od razu rozkład $P(Y=k|x)$. Jeśli spodziewamy się, że rozkłady warunkowe $p(x|Y=k)$ są rozkładami normalnymi, to model LDA może być lepszym wyborem niż regresja logistyczna. W przeciwnym przypadku wybieramy regresję logistyczną jako bardziej uniwersalny i mniej kosztowny obliczeniowo model.


W modelu QDA mamy

\begin{align*}
\log\frac{P(Y=k|x)}{P(Y=K|x)} &= a_k +\sum_{j=1}^p b_{kj}x_j + \sum_{j=1}^p\sum_{l=1}^p c_{kjl}x_jx_l,
\end{align*}

gdzie $a_k$, $b_{kj}$ i $c_{kjl}$ znów nie zależą od $x$ i są funkcjami parametrów modelu $P(Y=k)$, $\mu_k$ i $\Sigma_k$. Logarytmy szans w modelu QDA są funkcjami kwadratowymi względem $x$. 

W naiwnym klasyfikatorze Bayesa mamy

\begin{align*}
\log\frac{P(Y=k|x)}{P(Y=K|x)} &= \log\frac{P(Y=k)}{P(Y=K)} + \sum_{j=1}^p\log\frac{p(x_j|Y=k)}{p(x_j|Y=K)} \\
&= a_k +\sum_{j=1}^p g_{kj}(x_j),
\end{align*}

gdzie $a_k=\log(P(Y=k)/P(Y=K))$ i $g_{kj}(x_j)=\log(p(x_j|Y=k)/p(x_j|Y=K))$. 


Z powyższych przedstawień wynika:

* QDA sprowadza się do LDA, gdy $c_{kjl}=0$ dla wszystkich $k$, $j$ i $l$. Dzieje się tak dokładnie wtedy, gdy wszystkie macierze kowariancji $\Sigma_k$ są równe.
* Kładąc $g_{kj}(x_j)=b_{kj}x_j$ w naiwnym klasyfikatorze Bayesa otrzymujemy model LDA z niezależnymi cechami (macierz kowariancji $\Sigma$ jest diagonalna).
* QDA nie sprowadza się do naiwnego klasyfikatora Bayesa, ani naiwny klasyfikator Bayesa nie sprowadza się do QDA. W modelu Naive Bayes cechy są warunkowo niezależne, ale mogą być modelowana dowolnymi rozkładami. Z kolei w modelu QDA cechy mogą być skorelowane, ponadto model uwzględnia interakcje między nimi.